In [ ]:
import sys
import json

 #
 # These are the required imports for Fauna.
 #
from faunadb.client import FaunaClient
from faunadb import query as q

In [ ]:
# Set the parameters and connect to the local Fauna Instance

scheme = "http"
domain = "127.0.0.1"
port = "8443"
secret = "secret"
adminClient = FaunaClient(secret=secret, domain=domain, scheme=scheme, port=port)
print ("Connected to FaunaDB")

In [49]:
# Let us check if there are any existing databases

res = adminClient.query(q.paginate(q.databases()))
print (res)


{'data': [Ref(id=deba, class=Ref(id=databases)), Ref(id=Test, class=Ref(id=databases))]}


In [ ]:
# We could create a new database or go to the next step where we will drop the database if exists and recreate it
db_name = "Test"

res = adminClient.query(
        q.create_database({"name": db_name})
    )

print('DB {0} created: {1}'.format(db_name, res))

In [75]:
# We could have checked whether the database existed 
# Delete if exists and then create and if not just create it

res = adminClient.query(
        q.if_(
            q.exists(q.database(db_name)),
            [q.delete(q.database(db_name)), q.create_database({"name": db_name})],
            q.create_database({"name": db_name}))
    )

print('DB {0} created: {1}'.format(db_name, res))

DB Test created: [{'ref': Ref(id=Test, class=Ref(id=databases)), 'ts': 1534540314591513, 'name': 'Test'}, {'ref': Ref(id=Test, class=Ref(id=databases)), 'ts': 1534542291539347, 'name': 'Test'}]


In [76]:
# Now that the database is created let us create a key that will help us to connect to the DB

newKey = adminClient.query(q.select(["secret"], q.create_key({"database": q.database(db_name), "role": "server"})))
print('DB {0} secret: {1}'.format(db_name, newKey))

DB Test secret: fnAC4qnUGnACAMb59mI-vukrBm-Qo5qtgy907Cb2


In [77]:
# In this step we will create a connection to the newly created database using the key we just created above
client = FaunaClient(secret=newKey, domain="127.0.0.1", scheme="http", port=8443)

In [78]:
# Now let us create a new class called People

res = client.query(
        [
            q.create_class({"name": "People"})
        ]
    )
print('Created \'People\' class.')

Created 'People' class.


In [79]:
# The next step would be to create an index on the data
# Let us create an index on the state term

res = client.query([
        q.create_index({
            "name": "people_by_state",
            "source": q.class_("People"),
            "unique": False,
            "terms": {"field": ["data", "state"]}
        })
    ])

print('Created \'people_by_state\'index')

Created 'people_by_state'index


In [80]:
# In this section below we will create a bunch of fake data and load it into our class
# I am leveraging the Faker library to generate some synthetic data to populate the class
# You may change the number in the range field to increase or decrease 
# Once the data is populated, you can query it via the index or go to the dashboard and inspect it
#
from faker import Faker
import pandas as pd
import random
fake = Faker()
df = pd.DataFrame(columns=('name', 'email', 'address', 'city', 'state'))
for i in range(50):
    stuff = [fake.name()
        , fake.email()
        , fake.address()
        , fake.city()
        , fake.state()]
    df.loc[i] = [item for item in stuff]
    
json_data=df.to_json(orient='records')
js=json.loads(json_data)


res = client.query(
        q.map_(
            lambda People: q.create(q.class_("People"),
                                {"data": People}),
            js)
    )
print ("--End--")

--End--
